In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [3]:
file = open("/content/500Kwords.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines)

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:500]

'This page intentionally left blank crazy-readers.blogspot.com Artificial Intelligence A Modern Approach Third Edition crazy-readers.blogspot.com PRENTICE HALL SERIES IN ARTIFICIAL INTELLIGENCE Stuart Russell and Peter Norvig, Editors F ORSYTH & P ONCE G RAHAM J URAFSKY & M ARTIN N EAPOLITAN RUSSELL & N ORVIG Computer Vision: A Modern Approach ANSI Common Lisp Speech and Language Processing, 2nd ed. Learning Bayesian Networks Artificial Intelligence: A Modern Approach, 3rd ed. Artificial Intellig'

In [4]:
len(data)

502009

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[19, 473, 2959, 238, 865, 2960, 1321, 2961, 2962, 169, 144, 3, 631, 252, 1010]

In [6]:
len(sequence_data)

83681

In [7]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

7782
The Length of sequences are:  83678


array([[  19,  473, 2959,  238],
       [ 473, 2959,  238,  865],
       [2959,  238,  865, 2960],
       [ 238,  865, 2960, 1321],
       [ 865, 2960, 1321, 2961],
       [2960, 1321, 2961, 2962],
       [1321, 2961, 2962,  169],
       [2961, 2962,  169,  144],
       [2962,  169,  144,    3],
       [ 169,  144,    3,  631]])

In [8]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])

X = np.array(X)
y = np.array(y)

In [9]:
print("Data: ", X[:10])
print("Response: ", y[:10])

Data:  [[  19  473 2959]
 [ 473 2959  238]
 [2959  238  865]
 [ 238  865 2960]
 [ 865 2960 1321]
 [2960 1321 2961]
 [1321 2961 2962]
 [2961 2962  169]
 [2962  169  144]
 [ 169  144    3]]
Response:  [ 238  865 2960 1321 2961 2962  169  144    3  631]


In [10]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             77820     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 7782)              7789782   
                                                                 
Total params: 20916602 (79.79 MB)
Trainable params: 20916602 (79.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
from tensorflow import keras

from tensorflow.keras.utils import plot_model

# Assuming 'model' is your Keras model
plot_model(model, to_file='plot.png', show_layer_names=True)




You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=70, batch_size=64, callbacks=[checkpoint])

Epoch 1/70
1308/1308 [==============================] - ETA: 0s - loss: 6.8214
Epoch 1: loss improved from inf to 6.82142, saving model to next_words.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1308/1308 [==============================] - 120s 89ms/step - loss: 6.8214
Epoch 2/70
1308/1308 [==============================] - ETA: 0s - loss: 6.2539
Epoch 2: loss improved from 6.82142 to 6.25389, saving model to next_words.h5
1308/1308 [==============================] - 118s 90ms/step - loss: 6.2539
Epoch 3/70
1308/1308 [==============================] - ETA: 0s - loss: 5.8545
Epoch 3: loss improved from 6.25389 to 5.85447, saving model to next_words.h5
1308/1308 [==============================] - 117s 90ms/step - loss: 5.8545
Epoch 4/70
1308/1308 [==============================] - ETA: 0s - loss: 5.5262
Epoch 4: loss improved from 5.85447 to 5.52617, saving model to next_words.h5
1308/1308 [==============================] - 118s 90ms/step - loss: 5.5262
Epoch 5/70
1308/1308 [==============================] - ETA: 0s - loss: 5.2320
Epoch 5: loss improved from 5.52617 to 5.23201, saving model to next_words.h5
1308/1308 [==============================] - 118s 90ms/step - loss: 5.23

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def predict_next_words(model, tokenizer, text, num_words=10):
    sequence = tokenizer.texts_to_sequences([text])[0]
    predicted_words = []

    for _ in range(num_words):
        padded_sequence = np.array(sequence[-3:]).reshape(1, -1)  # Adjust depending on your input length
        preds = model.predict(padded_sequence)
        pred_word_index = np.argmax(preds, axis=-1)[0]
        predicted_word = tokenizer.index_word[pred_word_index] if pred_word_index in tokenizer.index_word else ''

        predicted_words.append(predicted_word)
        sequence.append(pred_word_index)  # Update sequence with the predicted index

    return ' '.join(predicted_words)

# Continuous prediction loop
while True:
    text_input = input("Enter your line: ")
    if text_input == "0":
        print("Execution completed.....")
        break
    else:
        try:
            print(predict_next_words(model, tokenizer, text_input, num_words=10))  # You can change num_words as needed
        except Exception as e:
            print("Error occurred: ", e)
            continue


Enter your line: what are agents
1/1 [==============================] - 0s 32ms/step
with certainty utility provides a way in which the agent
Enter your line: what is artifical intelligence
1/1 [==============================] - 0s 31ms/step
3 turing’s brain in the 1990s including the international workshop
Enter your line: what's the title of chapter 12
1/1 [==============================] - 0s 31ms/step
we show that any rational agent must behave as if
